# tf.data を使って CSV をロードする

https://www.tensorflow.org/tutorials/load_data/csv

In [1]:
import functools

import numpy as np
import tensorflow as tf

In [2]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

16384/13049 [=====================================] - 0s 0us/step


In [3]:
# numpy の値を読みやすくする
np.set_printoptions(precision=3, suppress=True)

In [5]:
!head {train_file_path}

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n


In [6]:
LABELS = [0, 1]
LABEL_COLUMN = 'survived'

In [7]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [8]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [9]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'male' b'female' b'male']
age                 : [29. 28. 28. 52. 36.]
n_siblings_spouses  : [1 0 0 1 0]
parch               : [0 0 0 0 0]
fare                : [66.6    7.896  6.858 78.267  7.496]
class               : [b'First' b'Third' b'Third' b'First' b'Third']
deck                : [b'C' b'unknown' b'unknown' b'D' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Queenstown' b'Cherbourg' b'Southampton']
alone               : [b'n' b'y' b'y' b'n' b'y']


In [10]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'female' b'female' b'female' b'male' b'male']
age                 : [28.  4. 22. 28. 16.]
n_siblings_spouses  : [1 0 0 0 0]
parch               : [0 2 0 0 0]
fare                : [24.15  22.025 10.517  7.75   9.5  ]
class               : [b'Third' b'Third' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'F' b'unknown']
embark_town         : [b'Queenstown' b'Southampton' b'Southampton' b'Queenstown' b'Southampton']
alone               : [b'n' b'n' b'y' b'y' b'y']


In [11]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [26. 36. 28. 17. 49.]
n_siblings_spouses  : [0 0 0 0 0]
class               : [b'Third' b'First' b'Third' b'Third' b'First']
deck                : [b'unknown' b'E' b'unknown' b'unknown' b'D']
alone               : [b'y' b'y' b'y' b'y' b'y']


In [12]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [28.  5. 22.  9. 25.]
n_siblings_spouses  : [0. 2. 1. 2. 1.]
parch               : [0. 1. 1. 2. 0.]
fare                : [30.5   19.258 29.    34.375  7.925]


In [13]:
example_batch, labels_batch = next(iter(temp_dataset))

In [14]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

In [15]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

[[ 28.      0.      0.      7.225]
 [ 33.      0.      0.      8.654]
 [ 25.      1.      0.      7.775]
 [ 28.      0.      0.      0.   ]
 [ 23.      1.      0.    113.275]]

[0 0 1 0 1]


In [16]:
show_batch(raw_train_data)

sex                 : [b'male' b'female' b'male' b'male' b'male']
age                 : [48. 36. 24. 21. 36.]
n_siblings_spouses  : [0 1 0 0 0]
parch               : [0 0 0 0 0]
fare                : [ 7.854 17.4    7.896  7.733 13.   ]
class               : [b'Third' b'Third' b'Third' b'Third' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Queenstown' b'Southampton']
alone               : [b'y' b'n' b'y' b'y' b'y']


In [17]:
example_batch, labels_batch = next(iter(temp_dataset))

In [18]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [19]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [20]:
show_batch(packed_train_data)

sex                 : [b'male' b'female' b'male' b'female' b'male']
class               : [b'First' b'Third' b'Second' b'First' b'Third']
deck                : [b'C' b'unknown' b'unknown' b'D' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Cherbourg' b'Southampton']
alone               : [b'n' b'y' b'y' b'n' b'y']
numeric             : [[ 19.      3.      2.    263.   ]
 [ 45.      0.      0.      7.75 ]
 [ 30.      0.      0.     13.   ]
 [ 23.      1.      0.    113.275]
 [ 22.      0.      0.      8.05 ]]


In [21]:
example_batch, labels_batch = next(iter(packed_train_data))

In [22]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [23]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [24]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

In [25]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7fcd2406e8b0>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [26]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[24.   ,  0.   ,  0.   ,  7.496],
       [38.   ,  0.   ,  0.   , 13.   ],
       [26.   ,  0.   ,  0.   ,  7.896],
       [ 2.   ,  1.   ,  1.   , 26.   ],
       [28.   ,  0.   ,  0.   ,  7.75 ]], dtype=float32)>

In [27]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-0.45 , -0.474, -0.479, -0.493],
       [ 0.669, -0.474, -0.479, -0.392],
       [-0.29 , -0.474, -0.479, -0.485],
       [-2.208,  0.395,  0.782, -0.154],
       [-0.13 , -0.474, -0.479, -0.488]], dtype=float32)

In [28]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [29]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [30]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [31]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


In [32]:
print(len(categorical_layer(example_batch).numpy()[0]))

20


In [33]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [34]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 1.     0.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.    -0.45  -0.474
 -0.479 -0.493  1.     0.   ]


In [35]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1),
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

In [36]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [37]:
model.fit(train_data, epochs=20)

Epoch 1/20
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
126/126 [==============================] - 1s 2ms/step - loss: 0.4944 - accuracy: 0.7544
Epoch 2/20
126/126 [==============================] - 0s 1ms/step - loss: 0.4199 - accuracy: 0.8341
Epoch 3/20
126/126 [==============================] - 0s 2ms/step - loss: 0.4010 - accuracy: 0.8325
Epoch 4/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3896 - accuracy: 0.8278
Epoch 5/20
126/126 [==============================] - 0s 1ms/step - loss: 0.3852 - accuracy: 0.8309
Epoch 6/20
126/126 [==============================] - 0s 1ms/step - loss: 0.3706 - accuracy: 0.8437
Epoch 7/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3679 - accuracy: 0.8453
Epoch 8/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3547 - accuracy: 0.8453
Epoch 9/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3558 - accurac

In [38]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

Consider rewriting this model with the Functional API.
53/53 [==============================] - 0s 1ms/step - loss: 0.4519 - accuracy: 0.8447


Test Loss 0.4518624544143677, Test Accuracy 0.8446969985961914


In [39]:
predictions = model.predict(test_data)

# 結果のいくつかを表示
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))

Consider rewriting this model with the Functional API.
Predicted survival: 115.09%  | Actual outcome:  SURVIVED
Predicted survival: -264.08%  | Actual outcome:  SURVIVED
Predicted survival: -168.39%  | Actual outcome:  DIED
Predicted survival: -168.39%  | Actual outcome:  DIED
Predicted survival: 542.30%  | Actual outcome:  DIED


In [44]:
bool(-0.1)

True